以下でimport
本来なら! git https://〜
でgithubからクローン

In [1]:
import numpy as np
import math
import pandas as pd
from scipy.interpolate import RegularGridInterpolator

# PIDの説明

In [2]:
# pid制御（プログラムの中身はd成分を無視したpi制御）
class PID:
    # def __init__()の中の値はデフォルト値。指定しなければこの値で計算される。
    def __init__(self, mode=1, a_max=1, a_min=0, kp=0.8, ti=10, t_reset=30, kg=1, sig=0, t_step=1):
        # mode          :運転時1, 非運転時0
        # a_max,a_min   :制御値の最大・最小範囲
        # kp            :比例ゲイン
        # ti            :積分時間
        # sv            :現時刻の制御する要素の設定値 (温度や圧力)  S0:前時刻(S0は不使用)
        # mv            :現時刻の制御する要素の値   P0:前時刻(P0は不使用)
        # sig           :積分総和
        # a             :現時刻の制御に使用する要素の値(周波数比、バルブ開度など)
        # ctrl          :制御に使用する要素の制御量
        # kg            :CTRLの量を制御する要素の設定値に落とし込むための係数。(-1か1)
        # t_reset       :設定値に不全がある、または不具合で制御が長時間おかしくなっていた場合にSIGをリセットするまでの時間
        # t_step        :制御ステップ。1だと毎時刻制御出力する、2だと2時刻ごとに制御出力する。
        # example kp=0.06,ti=20
        self.mode = mode
        self.a_max = a_max
        self.a_min = a_min
        self.kp = kp
        self.ti = ti
        self.flag_reset = np.zeros(t_reset)
        self.kg = kg
        self.sig = sig
        self.t_step = t_step
        self.t_step_cnt = -1 # 計算ステップのための内部パラメータ
        self.a = 0     
        
        
    def control(self, sv, mv):
        if self.mode == 1:
            
            self.t_step_cnt += 1
            if self.t_step_cnt % self.t_step == 0:
                self.t_step_cnt = 0
            
                self.sig += sv  - mv
                ctrl = self.kp * ((sv - mv) + 1 / self.ti * self.sig)
                
                # 前時刻からの偏差は5%未満とする
                if ctrl > 0.05:
                    ctrl = 0.05
                elif ctrl < -0.05:
                    ctrl = -0.05
                    
                self.a = self.a + ctrl * self.kg
        
                # 制御量は上下限値をもつ
                if self.a > self.a_max:
                    self.a = self.a_max
                elif self.a < self.a_min:
                    self.a = self.a_min
                    
                # 積分時間リセット
                t_reset = self.flag_reset.size
                for ii in range(t_reset - 1, 0, -1):
                    self.flag_reset[ii] = self.flag_reset[ii - 1]
                if sv - mv > 0:
                    self.flag_reset[0] = 1
                elif sv - mv < 0:
                    self.flag_reset[0] = -1
                else:
                    self.flag_reset[0] = 0
            
                if all(i == 1 for i in self.flag_reset) == True or all(i == -1 for i in self.flag_reset) == True:
                    self.sig = 0
                    self.flag_reset = np.zeros(t_reset)
            
            
        elif self.mode == 0:
            self.a = 0
            
        return self.a

以下でpidの設定値を入れる




In [3]:
pid = PID()
#以下で初期値を入れる
pid.mode = 1
pid.a_max = 1
pid.a_min = 0
pid.kp = 0.8
pid.ti = 10
pid.flag_reset = np.zeros(30)
pid.kg = 1
pid.sig = 0
pid.t_step = 1
pid.t_step_cnt = -1 

In [4]:
sv = 1000
mv = 150

In [5]:
print(pid.control(sv,mv))

0.05


# UnitNumの説明

In [8]:
class UnitNum:
    def __init__(self, thre_up=[0.5,1.0], thre_down=[0.4,0.9], t_wait=15):
        # thre_up_g     :増段閾値(1->2, 2->3といった時の値。型は配列またはリストとする) thre: threshold, g: 流量, q: 熱量
        # thre_down_q   :減段閾値(2->1, 3->2といった時の値。型は配列またはリストとする)
        # t_wait        :効果待ち時間(ex: 15分)
        # num           :運転台数 num: number
        # g             :流量[m3/min]
        # q             :熱量[kW]
        self.thre_up = thre_up
        self.thre_down = thre_down
        self.t_wait = t_wait
        self.num = 1
        self.flag_switch = np.zeros(t_wait)
        self.g = 0
            
    def control(self, g):
        self.g = g
        num_max = len(self.thre_up)+1
        
        for i in range(self.t_wait - 1, 0, -1):
            self.flag_switch[i] = self.flag_switch[i-1]
          
        if self.num == num_max:
            if self.g < self.thre_down[self.num-2]:
                self.flag_switch[0] = self.num-1
        elif self.num == 1:
            if self.g > self.thre_up[self.num-1]:
                self.flag_switch[0] = self.num+1
                
        elif self.g > self.thre_up[self.num-1]:
            self.flag_switch[0] = self.num+1
        elif self.g < self.thre_down[self.num-2]:
            self.flag_switch[0] = self.num-1
        else:
            self.flag_switch[0] = self.num
            
        if self.flag_switch[0] < 1:
            self.flag_switch[0] = 1
        elif self.flag_switch[0] > num_max:
            self.flag_switch[0] = num_max
        
            
        if all(i > self.num for i in self.flag_switch):
            self.num += 1
        elif all(i < self.num for i in self.flag_switch):
            self.num -= 1
        
        return self.num

以下でunitnumの設定値を入れる

In [10]:
unitnum = UnitNum()
unitnum.thre_up = [0.5,1.0]
unitnum.thre_down = [0.4,0.9]
unitnum.t_wait = 15
unitnum.num = 1
unitnum.flag_switch = np.zeros(unitnum.t_wait)

In [11]:
g = 100

In [12]:
print(unitnum.control(g))

1
